In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 1000)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [4]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 101.98it/s]


Total cost = 43076.54929261418

Cluster 71 assigned to Class 7. Cost: 122.06891530726965
Cluster 128 assigned to Class 16. Cost: 115.46425789725511
Cluster 154 assigned to Class 18. Cost: 107.32023481844182
Cluster 373 assigned to Class 19. Cost: 111.82501608497928
Cluster 80 assigned to Class 21. Cost: 118.8631354128029
Cluster 347 assigned to Class 22. Cost: 103.41628038377223
Cluster 83 assigned to Class 23. Cost: 110.85741815748486
Cluster 188 assigned to Class 51. Cost: 113.72646736714607
Cluster 361 assigned to Class 87. Cost: 129.63026405863013
Cluster 299 assigned to Class 88. Cost: 146.02221480875838
Cluster 335 assigned to Class 89. Cost: 108.66171759963082
Cluster 99 assigned to Class 93. Cost: 95.06951143730579
Cluster 93 assigned to Class 95. Cost: 113.25804710273474
Cluster 169 assigned to Class 96. Cost: 87.58723275275199
Cluster 195 assigned to Class 105. Cost: 92.43139639591783
Cluster 214 assigned to Class 112. Cost: 120.980524808103
Cluster 6 assigned to Class 117. C

Cluster 342 assigned to Class 869. Cost: 92.57889197723381
Cluster 242 assigned to Class 872. Cost: 98.4404543229766
Cluster 292 assigned to Class 875. Cost: 98.4836839471902
Cluster 276 assigned to Class 878. Cost: 121.24107753589509
Cluster 258 assigned to Class 879. Cost: 96.95330767457784
Cluster 196 assigned to Class 882. Cost: 126.71233145405957
Cluster 2 assigned to Class 887. Cost: 107.39205405048035
Cluster 322 assigned to Class 889. Cost: 95.27431340888596
Cluster 72 assigned to Class 891. Cost: 83.04046124000286
Cluster 371 assigned to Class 892. Cost: 184.56816707037464
Cluster 54 assigned to Class 893. Cost: 154.85802999578277
Cluster 176 assigned to Class 897. Cost: 92.91485989513254
Cluster 265 assigned to Class 902. Cost: 124.14143551917404
Cluster 274 assigned to Class 903. Cost: 91.61168188379106
Cluster 235 assigned to Class 906. Cost: 68.19560228658436
Cluster 184 assigned to Class 907. Cost: 89.06829991523273
Cluster 332 assigned to Class 909. Cost: 105.24383408932

In [5]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
